In [34]:
import pandas as pd
from slovnet import NER
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
from natasha import NewsNERTagger

Примерный план:
 - преобразовать текстовые данные в pandas 
 - найти именованные сущности, нормализовать, построить словари и сеты
 - 

## Преобразуем текстовые данные в датафрейм

In [35]:
file = open('data/raw/text_raw.txt', 'r', encoding = 'utf-8')
list_text = []
for line in file:
    line = line.strip()
    if line != '':
        list_text.append(line)

In [36]:
df_raw = pd.DataFrame({'text':list_text})

In [37]:
df_raw.isna().sum(), df_raw.isnull().sum()

(text    0
 dtype: int64,
 text    0
 dtype: int64)

Как видно от пропусков удалось избавится

In [38]:
df_raw.to_csv('data/interim/df_raw.csv', index=False)

Выделяем фичи из текстов

In [46]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

text = df_raw.loc[0, 'text']
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)

sent = doc.sents[0]

sent.syntax.print()

┌────────────► 17              obl
│              февраля         
│              1993            
│ ┌──────────► Постановлением  obl
│ │   ┌─┌──►┌─ Правительства   nmod
│ │   │ │   └► РФ              nmod
│ │   │ │   ┌► во              case
│ │ ┌►│ │ ┌─└─ исполнение      obl
│ │ │ │ └─└►┌─ Указа           nmod
│ │ │ │     └► Президента      nmod
│ │ │ └──────► РФ              nmod
│ │ │     ┌──► Государственный amod
│ │ │     │ ┌► газовый         amod
│ │ │ ┌►┌─└─└─ концерн         nsubj:pass
│ │ │ │ │   ┌► «               punct
│ │ │ │ └►┌─└─ Газпром         appos
│ │ │ │   └──► »               punct
└─└─└─└─────── преобразован    
  │ │   ┌────► в               case
  │ │   │ ┌──► Российское      amod
  │ │   │ │ ┌► акционерное     amod
  │ └──►└─└─└─ общество        obl
  └──────────► .               punct


In [47]:
doc.tag_ner(ner_tagger)

In [48]:
from natasha import NewsNERTagger
ner_tagger = NewsNERTagger(emb)
doc.tag_ner(ner_tagger)

doc.ner.print()

17 февраля 1993 Постановлением Правительства РФ во исполнение Указа 
                               ORG────────── LO                     
Президента РФ Государственный газовый концерн «Газпром» преобразован в
           LO                                  ORG────                
 Российское акционерное общество.
 ORG──────────────────────────── 


In [33]:
doc.text[31:44]

'Правительства'

In [26]:
doc.ner

NERMarkup(
    text='17 февраля 1993 Постановлением Правительства РФ во исполнение Указа Президента РФ Государственный газовый концерн «Газпром» преобразован в Российское акционерное общество.',
    spans=[Span(
         start=31,
         stop=44,
         type='ORG'
     ),
     Span(
         start=45,
         stop=47,
         type='LOC'
     ),
     Span(
         start=79,
         stop=81,
         type='LOC'
     ),
     Span(
         start=115,
         stop=122,
         type='ORG'
     ),
     Span(
         start=139,
         stop=170,
         type='ORG'
     )]
)